### Les librairies

In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from datetime import date
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re

### Mes fonctions

In [48]:

############## Mes fonctions ####################################

### ajouter un delay
def random_delay():
    time.sleep(random.uniform(2, 5))

### enlever les baslises html
def remove_html_tags(text_col):
    if text_col is None:
        return None
    else:
        clean = re.compile('<.*?>')
        return re.sub(clean,"", text_col)

### décompacter les tools
# def jtools_extractor(col):
#     tools = []                                      # je crée un liste pour stocker mes outils
#     for dicos in (df_jobs_temp["job.tools"][0]):     # je vais chercher chaque dico contenu dans le premier (et seul) item de ma liste de cette colonne
#         for key, tool in dicos.items():             # dans le duo key/value de mes items de chaque dico
#             if key == "name":                       # si la clé est = à name
#                 tools.append(dicos["name"])         # alors je stocke cet outil dans une liste
#     # return ", ".join(tools)                       # attention à bien remonter return en dehors de la deuxième boucle for
#     return(tools)

### décompacter les tools
def jtools_extractor(tools_list):
    tools = []                                      # je crée un liste pour stocker mes outils
    if isinstance(tools_list, list):                   # je vérifie que tools_list est une liste
        for dicos in tools_list:                       # je parcours chaque dictionnaire dans la liste des outils
            for key, tool in dicos.items():             # dans le duo key/value de mes items de chaque dico
                if key == "name":                       # si la clé est = à name
                    tools.append(dicos["name"])          # je stocke l'outil dans la liste
            if tools:  # je vérifie si des outils ont été ajoutées à la liste
                return tools
        else:
            return None  # je retourne None si aucun outil n'a été trouvé

### décompacter les skills
def jskills_extractor(skills_list):
    skills = []                                         # je crée une liste pour stocker les compétences
    if isinstance(skills_list, list):                   # je vérifie que skills_list est une liste
        for dictos in skills_list:                      # je parcours chaque dictionnaire dans la liste des compétences
            if 'name' in dictos:                        # je vérifie que la clé 'name' existe dans le dictionnaire
                value = dictos['name']
                if 'fr' in value:                       # je vérifie que la clé 'fr' existe dans le sous-dictionnaire 'name'
                    skills.append(value['fr'])          # je stocke la compétence dans la liste
            if skills:  # je vérifie si des compétences ont été ajoutées à la liste
                return skills
        else:
            return None  # je retourne None si aucune compétence en français n'a été trouvée
    else:
        return None  # je retourne None si skills_list n'est pas une liste

### récupérer ce qui est obligatoire pour la candidature
def appli_mandatory_extractor(mandatory_list):
    applications = []                                           # je crée un liste pour stocker mes éléments mandatory
    if isinstance(mandatory_list, list):                        # je vérifie que skills_list est une liste
        for dicts in mandatory_list:                          # je vais chercher chaque dico contenu dans cette liste
            for key, value in dicts.items():                    # dans le duo key/value de mes items de chaque dico
                if value == "mandatory":                        # si une value est = à mandatory
                    applications.append(dicts["name"])          # alors je stocke chaque clé correspondante à name
            if applications:  # je vérifie si des outils ont été ajoutées à la liste
                return applications
        else:
            return None  # je retourne None si aucun outil n'a été trouvé
    return(applications) 

### Procèdure

1. Aller sur la page de résultats avec selenium
2. Récupérer les URL pour aller de page en page
3. Faire une fonction pour aller récupérer les URL de chaque annonce sur chaque page
4. Couper les URL en relatif puis les reconcatener pour les passer dans l'API
5. Itérer pour requêter l'API avec chaque URL récupérée sur chaque page de résultats
6. Transformer en dataframe, décompacter et dropper ce qui doit l'être
7. Exporter en CSV avant l'EDA

### 1. Récupération des URL des pages et des jobs

Je commence par lancer le code pour récupérer les pages de résultats

In [49]:
############## SCRIPT PRINCIPAL COMPLET 1 ####################################
##############################################################################

## 1. je lance selenium pour aller sur la page de résultats ########################################################################
service = Service(executable_path = 'chromedriver.exe')                         # je dis où est mon driver                       
driver = webdriver.Chrome(service=service)                                      # je définis mon driver
driver.get("https://www.welcometothejungle.com/fr")                                                         # et je l'envoie sur la page d'accueil

wait = WebDriverWait(driver, 15)                                                 # je laisse le temps à la page de se charger

input_element = driver.find_element(By.ID,"search-query-field")                 # je sélectionne la barre de recherche
input_element.clear()                                                           # je la vide au cas où elle est pré-remplie                                             
input_element.send_keys("Data analyst" + Keys.ENTER)                            # j'écris dans la barre de recherche

random_delay()                                                                     # je laisse le temps à la page de charger

## 2. je vais chercher les url de pagination ######################################################################################
page_source = driver.page_source                                   # je vais chercher la page avec mon driver
soup = BeautifulSoup(page_source, "html.parser")                   #  je passe le code HTML à BeautifulSoup pour l'analyser

soup_pages = soup.find('nav', {"aria-label" : "Pagination"})      # Puis je vais chercher la section nav :

if soup_pages is not None:                                        # Je mets un if si la page n'est pas chargée et que nav est vide
    pli = soup_pages.find_all('li')                               # Puis je vais chercher tous les li contenus dans cette nav
    nb_pages = pli[-2].text                                       # Puis je vais chercher le texte de l'avant dernier li 
    print(f"\nJ'ai {nb_pages} pages")                               # (car le dernier c'est la flèche) pour avoir le nombre de pages

else:                                                             # si la page n'a pas eu le temps de charger, j'affiche un message
    print("Aucun élément HTML correspondant au sélecteur spécifié n'a été trouvé.")

result_url = driver.current_url                                   # je récupère l'URL sur laquelle je suis
base_url = result_url.replace("1","")

base_url = "https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page="
page_number = 1
urls_to_scrape = []

# 3. je fais une boucle qui itère le nom de la page autant de fois que nécessaire
for i in range(1,int(nb_pages)):
    url = f"{base_url}{i}"
    urls_to_scrape.append(url)
print(urls_to_scrape)

random_delay()     

jobs_list = []

for each_page in urls_to_scrape:
    try:
    # je charge la page actuelle dans le navigateur Web
      driver.get(each_page)
      random_delay() 

     # j'attends que la page soit entièrement chargée
      wait = WebDriverWait(driver, 20)
      wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, "Data")))

      links = (driver.find_elements(By.PARTIAL_LINK_TEXT, "Data")        # avec Selenium, je cherche les liens avec data dedans quelque soit la casse 
        or driver.find_elements(By.PARTIAL_LINK_TEXT, "DATA")
        or  driver.find_elements(By.PARTIAL_LINK_TEXT, "data"))

      urls = [link.get_attribute("href") for link in links]              # j'extraits le contenu de href pour le mettre dans la liste urls
      print(urls)
    
      jobs_list.extend(urls)
    except:
        print(f"An error occured on page {driver.current_url}")  

print(f"J'ai {len(jobs_list)} annonces au total")
print(jobs_list)

driver.quit()                                                     # je ferme le navigateur une fois tout fini



J'ai 34 pages
['https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=1', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=2', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=3', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=4', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffice

### 2. Récupération des infos de toutes les annonces

2.1 Formatage des URL pour l'API

In [50]:
## 1. formatage des urls en relative pour le requêtage ##

relative_urls = [sub.replace("https://www.welcometothejungle.com/fr/companies", "") for sub in jobs_list]
print(relative_urls)
print(len(relative_urls))                               # je vérifie que je n'ai rien perdu

## 2. formatage des urls relatives en url de requêtage ##
api_urls = []
for url in relative_urls:
    api_url = f"https://api.welcometothejungle.com/api/v1/organizations{url}"
    api_urls.append(api_url)
print(api_urls)
print(len(api_urls))                       # je vérifie que je n'ai rien perdu

['/grant-thornton/jobs/consultant-data-analytics-public-banque-neuilly-h-f_neuilly-sur-seine_GT_d8KYaJk?q=c5657079147e6f5a3f34bf2cfdb7e81c&o=996d2fc1-67b8-4d2d-ab9f-4ce584a48d33', '/thales/jobs/data-manager-projets-et-ingenierie-h-f_velizy-villacoublay?q=c5657079147e6f5a3f34bf2cfdb7e81c&o=7d44d4ad-a7a4-41da-800c-846db5c15c85', '/axa/jobs/data-analyst-f-h-cdi_malakoff?q=c5657079147e6f5a3f34bf2cfdb7e81c&o=a80f9927-da5b-4e53-887f-750c274dec67', '/stride-up/jobs/data-tracking-specialist_paris?q=c5657079147e6f5a3f34bf2cfdb7e81c&o=61f5c54e-f6bc-4819-bad9-4c201c312004', '/thelio/jobs/consultant-transformation-data?q=c5657079147e6f5a3f34bf2cfdb7e81c&o=b1a139bf-4ceb-4133-938f-e3520507fd4f', '/betclic/jobs/senior-gaming-data-analyst-h-f_bordeaux?q=c5657079147e6f5a3f34bf2cfdb7e81c&o=e3ca3399-9d84-4119-9522-1c8ee7c36b44', '/betclic/jobs/gaming-data-analyst-h-f_bordeaux_BG_9jx6XW1?q=c5657079147e6f5a3f34bf2cfdb7e81c&o=30e1bab9-9797-4be0-8489-2965717adbe5', '/ey/jobs/consultant-experimente-manager-en

2.2. Requêtage et fabrication du Dataframe

In [51]:
## 3.  je requête chaque URL et je stocke dans un dataframe ##

# création d'une liste vide pour stocker les données JSON
data_json = []

for job_url in api_urls:
    link = job_url
    r = requests.get(link)
    data_json.append(json.loads(r.text))                # comme j'itère sur chaque url je dois stocker ce résultat dans une liste
    # print(json.dumps(data_json, indent=2))

# j'en fais un dataframe
df_jraw = pd.json_normalize(data_json)
df_jraw

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
## 4. je nettoie et je formate ## 

# 4.1 je drop toutes les colonnes dont je n'ai pas besoin
df_jobs_temp = df_jraw.drop(columns=["job.urls", 
                                   "job.slug", 
                                   "job.videos", 
                                   "job.updated_at",
                                   "job.offices", 
                                   "job.is_default",
                                   'job.organization.labels',
                                   'job.organization.logo.url', 
                                   'job.organization.logo.thumb.url',
                                   'job.organization.sectors',
                                   'job.organization.cover_image.small.url',
                                   'job.organization.cover_image.url',
                                   'job.organization.cover_image.medium.url',
                                   'job.organization.cover_image.large.url',
                                   'job.organization.cover_image.social.url',
                                   'job.organization.website_organization.slug',
                                   'job.organization.website_organization.i18n_descriptions.fr',
                                   'job.organization.gdpr_setting.application_message',
                                   'job.organization.gdpr_setting.consent_duration',
                                   'job.organization.gdpr_setting.privacy_policy_url',
                                   'job.organization.headquarter.address',
                                   'job.organization.headquarter.local_address',
                                   'job.organization.headquarter.city',
                                   'job.organization.headquarter.latitude',
                                   'job.organization.headquarter.longitude',
                                   'job.organization.headquarter.district',
                                   'job.organization.headquarter.country_code',
                                   'job.organization.headquarter.zip_code',
                                   'job.organization.headquarter.local_city',
                                   'job.organization.headquarter.local_district',
                                   'job.organization.media_facebook',
                                   'job.organization.media_instagram',
                                   'job.organization.media_linkedin',
                                   'job.organization.media_pinterest',
                                   'job.organization.media_twitter',
                                   'job.organization.media_youtube',
                                   'job.organization.playlist_id',
                                   'job.organization.video_playlist_provider',
                                   'job.organization.automatic_email',
                                   'job.archived_at',
                                   'job.profession.name.cs', 'job.profession.name.en',
                                   'job.profession.name.es', 'job.profession.name.sk',
                                   'job.profession.category.cs',
                                   'job.profession.category.en', 'job.profession.category.es',
                                   'job.profession.category.fr', 'job.profession.category.sk',
                                   'job.profession.category_name.cs',
                                   'job.profession.category_name.en',
                                   'job.profession.category_name.es',
                                   'job.profession.category_name.sk',
                                   'job.profession.sub_category_name.cs',
                                   'job.profession.sub_category_name.en',
                                   'job.profession.sub_category_name.es',
                                   'job.profession.sub_category_name.sk',
                                   'job.profession.sub_category_reference',
                                   'job.questions',
                                   "job.featured_page.type",
                                   'job.featured_page.slug',
                                   'job.cta_content.links',
                                   'job.cta_content.contents',
                                   'job.organization.website_organization.i18n_descriptions.en',
                                   'job.organization.website_organization.i18n_descriptions.es'
                                   ]
                                   )



### Les fonctions

In [ ]:
### décompacter les tools
# def jtools_extractor(col):
#     tools = []                                      # je crée un liste pour stocker mes outils
#     for dicos in (df_jobs_temp["job.tools"][0]):     # je vais chercher chaque dico contenu dans le premier (et seul) item de ma liste de cette colonne
#         for key, tool in dicos.items():             # dans le duo key/value de mes items de chaque dico
#             if key == "name":                       # si la clé est = à name
#                 tools.append(dicos["name"])         # alors je stocke cet outil dans une liste
#     # return ", ".join(tools)                       # attention à bien remonter return en dehors de la deuxième boucle for
#     return(tools)

# def appli_mandatory_extractor(col):
#     applications = []                                           # je crée un liste pour stocker mes éléments mandatory
#     for dicts in (df_jobs_temp["job.application_fields"][0]):    # je vais chercher chaque dico dans cette liste
#             for key, value in dicts.items():                    # dans le duo key/value de mes items de chaque dico
#                 if value == "mandatory":                        # si une value est = à mandatory
#                     applications.append(dicts["name"])          # alors je stocke chaque clé correspondante à name
#     return(applications)   



In [ ]:
############## Mes fonctions ####################################

### ajouter un delay
def random_delay():
    time.sleep(random.uniform(2, 5))


### enlever les baslises html
def remove_html_tags(text_col):
    if text_col is None:
        return None
    else:
        clean = re.compile('<.*?>')
        return re.sub(clean,"", text_col)
    

### convertir l'année de création en année
def convert_year_to_date(value):
    try:
        # Convertir la valeur en int, puis en string et enfin en datetime avec le format %Y
        year_str = str(int(value))
        return pd.to_datetime(year_str, format='%Y').strftime('%Y')
    except:                                                                       # Je retourne la valeur originale si ce n'est pas une date
      return value


### remplacer les catégories de niveau d'expérience
def experience_replace(experience):
    lower_exp = experience.lower()
    small_exp = lower_exp.replace("_", " ")
    return(small_exp)

### décompacter les tools
def jtools_extractor(tools_list):
    tools = []                                      # je crée un liste pour stocker mes outils
    if isinstance(tools_list, list):                   # je vérifie que tools_list est une liste
        for dicos in tools_list:                       # je parcours chaque dictionnaire dans la liste des outils
            for key, tool in dicos.items():             # dans le duo key/value de mes items de chaque dico
                if key == "name":                       # si la clé est = à name
                    tools.append(dicos["name"])          # je stocke l'outil dans la liste
            if tools:  # je vérifie si des outils ont été ajoutées à la liste
                return tools
        else:
            return None  # je retourne None si aucun outil n'a été trouvé


### décompacter les skills
def jskills_extractor(skills_list):
    skills = []                                         # je crée une liste pour stocker les compétences
    if isinstance(skills_list, list):                   # je vérifie que skills_list est une liste
        for dictos in skills_list:                      # je parcours chaque dictionnaire dans la liste des compétences
            if 'name' in dictos:                        # je vérifie que la clé 'name' existe dans le dictionnaire
                value = dictos['name']
                if 'fr' in value:                       # je vérifie que la clé 'fr' existe dans le sous-dictionnaire 'name'
                    skills.append(value['fr'])          # je stocke la compétence dans la liste
            if skills:  # je vérifie si des compétences ont été ajoutées à la liste
                return skills
        else:
            return None  # je retourne None si aucune compétence en français n'a été trouvée
    else:
        return None  # je retourne None si skills_list n'est pas une liste

### récupérer ce qui est obligatoire pour la candidature


def appli_mandatory_extractor(mandatory_list):
    applications = []                                           # je crée un liste pour stocker mes éléments mandatory
    if isinstance(mandatory_list, list):                        # je vérifie que skills_list est une liste
        for dicts in mandatory_list:                          # je vais chercher chaque dico contenu dans cette liste
            for key, value in dicts.items():                    # dans le duo key/value de mes items de chaque dico
                if value == "mandatory":                        # si une value est = à mandatory
                    applications.append(dicts["name"])          # alors je stocke chaque clé correspondante à name
            if applications:  # je vérifie si des outils ont été ajoutées à la liste
                return applications
        else:
            return None  # je retourne None si aucun outil n'a été trouvé
    return(applications) 

In [ ]:
# 4.2 je passe toutes mes fonctions

# html cleaner
### faire un if none pass sinon
df_jobs_temp[["job.organization.description_clean","job.profile_clean"]] = df_jobs_temp[["job.organization.description","job.profile"]].map(remove_html_tags).copy()

# level bins cleaner
df_jobs_temp["job.experience_level_clean"] = df_jobs_temp["job.experience_level"].astype(str).apply(lambda x: experience_replace(x))
df_jobs_temp["job.education_level_clean"] = df_jobs_temp["job.education_level"].astype(str).apply(lambda x: experience_replace(x))

# date convertor
df_jobs_temp['job.organization.creation_year_date'] = df_jobs_temp['job.organization.creation_year'].apply(convert_date)    

# tools extractor
df_jobs_temp["job.tools_clean"] = df_jobs_temp["job.tools"].apply(jtools_extractor)

# skills extractor
df_jobs_temp["job.skills_clean"] = df_jobs_temp["job.skills"].apply(jskills_extractor)

# application mandatories
df_jobs_temp["job.application_mandatory"] = df_jobs_temp["job.application_fields"].apply(appli_mandatory_extractor)

In [ ]:
df_jobs_temp["job.skills_clean"].head(20)

0                      [Contrôle qualité]
1                                    None
2        [Collecte et analyse de données]
3                   [Gestion des données]
4                                    None
5                                    None
6                                    None
7                                    None
8     [Collaboration et travail d'équipe]
9                                    None
10                                   None
11                                   None
12                   [Analyse financière]
13                                   None
14      [Réglementation environnementale]
15                                   None
16                                   None
17            [Visualisation des données]
18                                   None
19                                   None
Name: job.skills_clean, dtype: object

### Je continue de nettoyer

In [ ]:
# 4.3 puis je drop les colonnes que j'ai nettoyé
df_jobs = df_jobs_temp.drop(columns=["job.skills", "job.tools", "job.application_fields", "job.profile","job.organization.description_clean", "job.organization.creation_year"])

### J'exporte en csv

In [ ]:
# 5. j'exporte en csv avec date et heure du moment
# today = date.today()
now = datetime.now()

# Étape 2: je formate la date dd-mm-hh-mn
formatted_date = now.strftime("%d-%m_%H-%M")

# Étape 3: j'ajoute la date au nom de fichier
file_name = f"wttj_{formatted_date}.csv"

df_jobs.to_csv(file_name, index=False)

In [ ]:
print(now)

2024-06-24 11:23:09.926771


In [ ]:
df_jobs.sample(3)

,job.contract_duration_min,job.office.address,job.office.local_address,job.office.city,job.office.latitude,job.office.longitude,job.office.district,job.office.country_code,job.office.zip_code,job.office.local_city,...,job.organization.gdpr_setting,job.organization.website_organization.i18n_descriptions.en,job.benefits.FR.categories,job.benefits.FR.count,job.benefits.FR.preview,job.organization.website_organization.i18n_descriptions.es,job.profile_clean,job.tools_clean,job.skills_clean,job.application_mandatory
75,NaN,"16 Rue de la Ville-l'Évêque, 75008 Paris, France",None,Paris,48.87160,2.32037,Paris,FR,75008,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Compétences humaines &amp; techniques nécessai...,None,None,[cover_letter]
267,NaN,"36 Rue de Saint-Pétersbourg, 75008 Paris, France","36 Rue de Saint-Pétersbourg, 75008 Paris, France",Paris,48.88262,2.32644,Paris,FR,75008,Paris,...,NaN,NaN,NaN,NaN,NaN,NaN,Pourquoi êtes-vous notre TOP candidat ? 🧐Vous ...,[Sql],None,[cover_letter]
68,NaN,"44 Rue Traversière, 92100 Boulogne-Billancourt...",None,Boulogne-Billancourt,48.82458,2.24270,Hauts-de-Seine,FR,92100,None,...,NaN,NaN,NaN,NaN,NaN,NaN,De formation Bac+4/5 en Statistiques ou Data A...,[Scala],[Outils d'automatisation du marketing],[resume]


In [ ]:
df_jobs.columns

Index(['job.contract_duration_min', 'job.office.address',
       'job.office.local_address', 'job.office.city', 'job.office.latitude',
       'job.office.longitude', 'job.office.district',
       'job.office.country_code', 'job.office.zip_code',
       'job.office.local_city', 'job.office.local_district',
       'job.profession.name.fr', 'job.profession.category_reference',
       'job.profession.category_name.fr',
       'job.profession.sub_category_name.fr', 'job.remote', 'job.reference',
       'job.published_at', 'job.start_date', 'job.salary_currency',
       'job.experience_level', 'job.education_level', 'job.salary_min',
       'job.salary_period', 'job.name', 'job.organization.has_external_ats',
       'job.organization.reference', 'job.organization.nb_employees',
       'job.organization.parity_men', 'job.organization.revenue',
       'job.organization.profile_type', 'job.organization.name',
       'job.organization.average_age', 'job.organization.default_language',
       'jo